In [1]:
import numpy as np
import pandas as pd
import os
import glob
from collections import Counter
from itertools import groupby

pd.set_option('display.max_columns', None)

# it's not best practice to read these columns in as objects just to silence the dtype
# warning but I really don't care about these columns and won't use them anyway
RAM10 = pd.read_csv('RAM-10_raw_data.csv', dtype={'Min': object})
RAM11 = pd.read_csv('RAM-11_raw_data.csv', dtype={'Min': object, 'Indi_var_group': object})


# dropping this Coord column right now because it's read in as strings
# when I want it to be tuples of floats for nearest neighbor calculation
# i'll toss in another coord column later
RAM10 = RAM10.drop(columns=['Coord'])

# dropping these columns because they are useless
RAM10 = RAM10.drop(columns=['Unnamed: 0','Ind_var_group', 'Indi_var_group'])
RAM11 = RAM11.drop(columns=['Unnamed: 0','Ind_var_group', 'Indi_var_group'])

# reordering columns so RAM10 and RAM11 match for concat
RAM10.columns = [' ', 'AR', 'AnalysisDate', 'Area', 'Background', 'CellNumber', 'Circ.',
       'Experiment', 'FileName', 'Integrated_Density', 'Max', 'Mean',
       'Mean-Background', 'Min', 'Round', 'Solidity', 'Stain', 'StdDev',
       'Subject', 'XM', 'YM', 'directory', 'roiName', 'Rat_n', 'Treatment',
       'Dox', 'React']


# check that stain type names are consistent
print(np.unique(RAM10.Stain) == np.unique(RAM11.Stain))

# standardizing Rat_n string format
rats = list(RAM10.Rat_n)
rats = ['_'.join(['RAM-10', rat]) for rat in rats]
RAM10['Rat_n'] = rats

RAM10_col = set(RAM10.columns)
RAM11_col = set(RAM11.columns)

print(f'n of RAM-10 columns: {len(RAM10_col)}')
print(f'n of RAM-11 columns: {len(RAM11_col)}')

print(f'Symmetric difference of RAM-10 and RAM-11 columns: {RAM11_col.symmetric_difference(RAM10_col)}')
print(f'RAM-10 columns are equal to RAM-11 columns: {RAM11_col == RAM10_col}')

# great! now that I've confirmed that the sets are standardized, it is safe to concat.

[ True  True  True  True  True  True  True  True  True  True  True  True]
n of RAM-10 columns: 27
n of RAM-11 columns: 27
Symmetric difference of RAM-10 and RAM-11 columns: set()
RAM-10 columns are equal to RAM-11 columns: True


In [2]:
# concat!

RAM1011 = pd.concat([RAM10,RAM11], axis=0, sort=False)
RAM1011['Coord'] = list(zip(RAM1011.XM, RAM1011.YM))
RAM1011 = RAM1011.dropna()

RAM1011

,,AR,AnalysisDate,Area,Background,CellNumber,Circ.,Experiment,FileName,Integrated_Density,Max,Mean,Mean-Background,Min,Round,Solidity,Stain,StdDev,Subject,XM,YM,directory,roiName,Rat_n,Treatment,Dox,React,Coord
0,1,1.287,_11_20_2022_at_15-35,23.148,39.644,1,0.267,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,249.431,73.808,50.420,10.776,41.869,0.204,0.266,c-Fos_coloc_w/_EGFP,6.707,10C,30.322,16.611,/Volumes/T7/RAM-10/summed/double-label_results...,0020a-0035,RAM-10_rat10C,ChABC,DOX_ON,NR,"(30.322, 16.611)"
1,2,1.169,_11_20_2022_at_15-35,18.939,39.644,2,0.283,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,288.216,100.335,54.862,15.218,39.854,0.241,0.276,c-Fos_coloc_w/_EGFP,14.585,10C,141.285,29.432,/Volumes/T7/RAM-10/summed/double-label_results...,0036a-0169,RAM-10_rat10C,ChABC,DOX_ON,NR,"(141.285, 29.432)"
2,3,1.209,_11_20_2022_at_15-35,70.846,39.644,3,0.890,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,2820.040,141.939,79.449,39.805,39.711999999999996,0.727,0.894,c-Fos_coloc_w/_EGFP,25.539,10C,54.411,393.511,/Volumes/T7/RAM-10/summed/double-label_results...,005a-00000,RAM-10_rat10C,ChABC,DOX_ON,NR,"(54.411, 393.511)"
3,4,1.000,_11_20_2022_at_15-35,67.338,39.644,4,1.000,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,4351.428,185.912,104.264,64.620,42.669,0.990,0.950,c-Fos_coloc_w/_EGFP,32.926,10C,390.051,229.962,/Volumes/T7/RAM-10/summed/double-label_results...,005a-00001,RAM-10_rat10C,ChABC,DOX_ON,NR,"(390.051, 229.962)"
4,5,1.000,_11_20_2022_at_15-35,68.040,39.644,5,1.000,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,6193.684,264.734,130.674,91.030,50.056999999999995,1.000,0.960,c-Fos_coloc_w/_EGFP,44.663,10C,211.344,302.819,/Volumes/T7/RAM-10/summed/double-label_results...,005a-00002,RAM-10_rat10C,ChABC,DOX_ON,NR,"(211.344, 302.819)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35380,48,1.169,_12_15_2022_at_10-54,52.608,18.626,48,0.786,eg:Exp1,RAM-11_11_PL_3.7_C_4,969.699,65.909,37.059,18.432,19.514,0.668,0.765,mKate2,11.765,eg:Sub_42B,140.687,342.930,RAM-11_PFC/summed/RAM-11_11_PL_3.7_C_4/RAM-11_...,0410c-0169,RAM-11_rat11,Vehicle,DOX_OFF,NR,"(140.687, 342.93)"
35381,49,1.435,_12_15_2022_at_10-54,33.669,18.626,49,0.497,eg:Exp1,RAM-11_11_PL_3.7_C_4,348.930,54.830,28.989,10.363,19.642,0.376,0.527,mKate2,8.455,eg:Sub_42B,123.218,346.115,RAM-11_PFC/summed/RAM-11_11_PL_3.7_C_4/RAM-11_...,0413c-0147,RAM-11_rat11,Vehicle,DOX_OFF,NR,"(123.218, 346.115)"
35382,50,1.092,_12_15_2022_at_10-54,49.101,18.626,50,0.986,eg:Exp1,RAM-11_11_PL_3.7_C_4,19696.145,844.305,419.762,401.136,33.643,0.916,0.946,mKate2,218.974,eg:Sub_42B,157.981,343.253,RAM-11_PFC/summed/RAM-11_11_PL_3.7_C_4/RAM-11_...,0410c-0188,RAM-11_rat11,Vehicle,DOX_OFF,NR,"(157.981, 343.25300000000004)"
35383,51,1.209,_12_15_2022_at_10-54,80.666,18.626,51,1.000,eg:Exp1,RAM-11_11_PL_3.7_C_4,70964.972,1607.205,898.367,879.741,32.195,0.827,1.018,mKate2,455.514,eg:Sub_42B,162.717,335.693,RAM-11_PFC/summed/RAM-11_11_PL_3.7_C_4/RAM-11_...,0400c-0194,RAM-11_rat11,Vehicle,DOX_OFF,NR,"(162.717, 335.69300000000004)"


In [3]:
# ok now that the sets are standardized, I'll have to split them again to normalize the intensity separately.

RAM10 = RAM1011[RAM1011.Rat_n.str.contains('RAM-10')]
RAM11 = RAM1011[RAM1011.Rat_n.str.contains('RAM-11')]

RAM10_bystain = [RAM10[RAM10['Stain'] == stain] for stain in np.unique(RAM10.Stain)]
RAM11_bystain = [RAM11[RAM11['Stain'] == stain] for stain in np.unique(RAM11.Stain)]


def drop_dups_norm(lst_df_by_stain, normcondition1, normcondition2):
    
    ### dropping duplicates ###
    lst_df_dropped_duplicates = []

    for df in lst_df_by_stain:
        lst_df_dropped_duplicates.append(
            df.drop_duplicates(subset=['Area', 'Background', 'CellNumber', 'Circ.',
           'Experiment', 'FileName', 'Integrated_Density', 'Max', 'Mean','Min', 'Round',
           'Solidity', 'Stain', 'StdDev', 'Subject', 'XM', 'YM', 'roiName', 'Coord'])
        )   
            ##### PLEASE CHECK THESE COLUMN NAMES ##### 
            # (should be fine for most up to date pipsqueak versions)

    ### Normalize ###
    norm_means = []
    lst_df_by_stain_dropped_duplicates_NORM = []

    for df in lst_df_dropped_duplicates:
        # grabbing the mean of our norm condition
        df_norm = df[(df['Dox'] == normcondition1) & (df['React'] == normcondition2)]  ### norm condition
        norm_mean = df_norm['Mean-Background'].mean()
        norm_means.append(norm_mean)

        # dividing our Mean-Background by Veh mean
        df_copy = df
        df_copy['Mean-Background_NORM'] = df['Mean-Background'] / norm_mean

        # storing new DataFrame in a list
        lst_df_by_stain_dropped_duplicates_NORM.append(df_copy)
    
    # check that the mean of the normalized data is = to 1 (or something very close)
    for i, df in enumerate(lst_df_by_stain_dropped_duplicates_NORM):
        a = df[(df['Dox'] == normcondition1) & (df['React'] == normcondition2)]
        print(f"[{i}] {np.unique(a.Stain).item()}: {np.average(a['Mean-Background_NORM'])}")

    ### export csvs ###
    #file_names_norm = [name.replace('.csv','_NORM.csv') for name in csv_names]

    #for i in range(len(file_names_norm)):
    #    lst_df_by_stain_dropped_duplicates_NORM[i].to_csv(file_names_norm[i])
        
    return lst_df_by_stain_dropped_duplicates_NORM

RAM10_bystain_NORM = drop_dups_norm(RAM10_bystain, 'DOX_ON', 'FR1')
RAM11_bystain_NORM = drop_dups_norm(RAM11_bystain, 'DOX_ON', 'FR1')

[0] EGFP: 0.9999999999999997
[1] EGFP_coloc_w/_c-Fos: 0.9999999999999999
[2] EGFP_coloc_w/_mKate2: 0.9999999999999997
[3] Triple-labeled_EGFP: 1.0
[4] Triple-labeled_c-Fos: 0.9999999999999998
[5] Triple-labeled_mKate2: 1.0
[6] c-Fos: 0.999999999999999
[7] c-Fos_coloc_w/_EGFP: 1.000000000000001
[8] c-Fos_coloc_w/_mKate2: 1.0000000000000002
[9] mKate2: 1.0000000000000002
[10] mKate2_coloc_w/_EGFP: 1.0000000000000002
[11] mKate2_coloc_w/_c-Fos: 1.0
[0] EGFP: 0.9999999999999997
[1] EGFP_coloc_w/_c-Fos: 0.9999999999999996
[2] EGFP_coloc_w/_mKate2: 1.0000000000000002
[3] Triple-labeled_EGFP: 1.0
[4] Triple-labeled_c-Fos: 1.0
[5] Triple-labeled_mKate2: 1.0
[6] c-Fos: 1.0
[7] c-Fos_coloc_w/_EGFP: 1.0000000000000004
[8] c-Fos_coloc_w/_mKate2: 1.0
[9] mKate2: 0.9999999999999996
[10] mKate2_coloc_w/_EGFP: 1.0
[11] mKate2_coloc_w/_c-Fos: 1.0000000000000002


In [4]:
RAM10_cfos_median = np.median(RAM10_bystain_NORM[6]['Mean-Background_NORM'])
RAM11_cfos_median = np.median(RAM11_bystain_NORM[6]['Mean-Background_NORM'])

RAM10_cfos = RAM10_bystain_NORM[6]
RAM11_cfos = RAM11_bystain_NORM[6]

RAM10_cfosonmkate = RAM10_bystain_NORM[8]
RAM11_cfosonmkate = RAM11_bystain_NORM[8]

RAM10_mkateoncfos = RAM10_bystain_NORM[11]
RAM11_mkateoncfos = RAM11_bystain_NORM[11]

print(RAM10_cfos_median)
print(RAM11_cfos_median)

1.0592548336237295
1.0286025432410617


In [5]:
# ok let's just focus on RAM 8 for now then repeat whatever we do for RAM 9

def new_filenames(df):
    filenames = []
    for rat_n, filename in zip(list(df['Rat_n']), list(df['FileName'])):
        f = '_'.join([rat_n, '_'.join(filename.split('_')[2:])])         # Here I just wanted to add explicit
        f = '_'.join(f.split('_')[:-1])                                  # rat_ns exactly as I have them in the
                                                                         # rat_n column
        filenames.append(f)
        
    df['FileName'] = filenames
    
    return df

RAM10_cfos = new_filenames(RAM10_cfos)
RAM10_cfosonmkate = new_filenames(RAM10_cfosonmkate)
RAM10_mkateoncfos = new_filenames(RAM10_mkateoncfos)
RAM11_cfos = new_filenames(RAM11_cfos)
RAM11_cfosonmkate = new_filenames(RAM11_cfosonmkate)
RAM11_mkateoncfos = new_filenames(RAM11_mkateoncfos)

In [6]:
# check that we have the same image names across stain types
print(set(RAM10_cfos.FileName).issuperset(set(RAM10_mkateoncfos.FileName)))

def distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

images = np.unique(RAM10_cfosonmkate.FileName)


mkateoncfos_nearest_cfos = []
for image in images:
    # making copies to mess around with (that are also more conveniently named)
    cfos = RAM10_cfos[RAM10_cfos.FileName == image].copy()
    mkateoncfos = RAM10_mkateoncfos[RAM10_mkateoncfos.FileName == image].copy()
    
    cfos_ints = []
    nearest_cfos_coord = []
    for coord1 in mkateoncfos.Coord:
        
        # calculate distances between a given mkate_double cell and every other c-fos cell in that image
        coords_d = [(coord2, distance(coord1, coord2)) for coord2 in list(cfos.Coord)]
        
        # find the c-fos cell that yields the shortest distance to the given mkate_double cell
        min_coord, min_d = min(coords_d, key=lambda a:a[1])
        
        # get the intensity of that c-fos cell
        cfos_int = np.array(cfos[cfos.Coord == min_coord]['Mean-Background_NORM'])[0]
        
        # toss into list
        cfos_ints.append(cfos_int)
        nearest_cfos_coord.append(min_coord)
    
    # toss into dataframe
    mkateoncfos['Nearest_cfos_coord'] = nearest_cfos_coord
    mkateoncfos['Nearest_cfos_int'] = cfos_ints
    
    mkateoncfos_nearest_cfos.append(mkateoncfos)
    
RAM10_mkateoncfos_nearestcfos = pd.concat(mkateoncfos_nearest_cfos)

True


In [7]:
# same for RAM11
images = np.unique(RAM11_cfosonmkate.FileName)


mkateoncfos_nearest_cfos = []
for image in images:
    cfos = RAM11_cfos[RAM11_cfos.FileName == image].copy()
    mkateoncfos = RAM11_mkateoncfos[RAM11_mkateoncfos.FileName == image].copy()
    
    cfos_ints = []
    nearest_cfos_coord = []
    for coord1 in mkateoncfos.Coord:
        coords_d = [(coord2, distance(coord1, coord2)) for coord2 in list(cfos.Coord)]
        min_coord, min_d = min(coords_d, key=lambda a:a[1])
        
        cfos_int = np.array(cfos[cfos.Coord == min_coord]['Mean-Background_NORM'])[0]
        cfos_ints.append(cfos_int)
        
        nearest_cfos_coord.append(min_coord)
        
    mkateoncfos['Nearest_cfos_coord'] = nearest_cfos_coord
    mkateoncfos['Nearest_cfos_int'] = cfos_ints
    
    mkateoncfos_nearest_cfos.append(mkateoncfos)
    
RAM11_mkateoncfos_nearestcfos = pd.concat(mkateoncfos_nearest_cfos)

In [8]:
RAM10_nearest_cfos = RAM10_mkateoncfos_nearestcfos['Nearest_cfos_int']
RAM11_nearest_cfos = RAM11_mkateoncfos_nearestcfos['Nearest_cfos_int']

# median split into c-Fos high and c-Fos low
cfos_highlow = []
for cfos_int in RAM10_nearest_cfos:
    if cfos_int < RAM10_cfos_median:
        cfos_highlow.append('c-Fos low')
    elif cfos_int >= RAM10_cfos_median:
        cfos_highlow.append('c-Fos high')
RAM10_mkateoncfos_nearestcfos['c-Fos Intensity Bin'] = cfos_highlow

# same for RAM11
cfos_highlow = []
for cfos_int in RAM11_nearest_cfos:
    if cfos_int < RAM11_cfos_median:
        cfos_highlow.append('c-Fos low')
    elif cfos_int >= RAM11_cfos_median:
        cfos_highlow.append('c-Fos high')
RAM11_mkateoncfos_nearestcfos['c-Fos Intensity Bin'] = cfos_highlow

RAM1011_mkatedouble_cfoshighlow = pd.concat([RAM10_mkateoncfos_nearestcfos, RAM11_mkateoncfos_nearestcfos])

In [9]:
df = RAM1011_mkatedouble_cfoshighlow 

cfos_high_by_image = []
cfos_low_by_image = []
images = []
for rat in np.unique(df['Rat_n']):
    df_rat = df[df.Rat_n == rat]
    
    for image in np.unique(df_rat['FileName']):
        df_rat_image = df_rat[df_rat.FileName == image]
        cfos_high_by_image.append(len(df_rat_image[df_rat_image['c-Fos Intensity Bin'] == 'c-Fos high']))
        cfos_low_by_image.append(len(df_rat_image[df_rat_image['c-Fos Intensity Bin'] == 'c-Fos low']))
        images.append(image)

RAM1011_mkatedouble_cfoshighlow_counts = pd.DataFrame()
RAM1011_mkatedouble_cfoshighlow_counts['Image'] = images
RAM1011_mkatedouble_cfoshighlow_counts['mKate2 on c-Fos counts, c-Fos high'] = cfos_high_by_image
RAM1011_mkatedouble_cfoshighlow_counts['mKate2 on c-Fos counts, c-Fos low'] = cfos_low_by_image

In [10]:
rat_ns = []
for image in RAM1011_mkatedouble_cfoshighlow_counts['Image']:
    rat_n = '_'.join(image.split('_')[0:2])
    rat_ns.append(rat_n)
    
RAM1011_mkatedouble_cfoshighlow_counts['Rat_n'] = rat_ns

rats = np.unique(RAM1011.Rat_n)

SUM_mkatedouble_cfos_high = []
SUM_mkatedouble_cfos_low = []
rat_ns = []
for rat in rats:
    
    if rat in set(RAM1011_mkatedouble_cfoshighlow_counts['Rat_n']):
        df = RAM1011_mkatedouble_cfoshighlow_counts[RAM1011_mkatedouble_cfoshighlow_counts['Rat_n'] == rat]
        SUM_mkatedouble_cfos_high.append(np.sum(df['mKate2 on c-Fos counts, c-Fos high']))
        SUM_mkatedouble_cfos_low.append(np.sum(df['mKate2 on c-Fos counts, c-Fos low']))
        rat_ns.append(rat)
        
    elif rat not in set(RAM1011_mkatedouble_cfoshighlow_counts['Rat_n']):
        SUM_mkatedouble_cfos_high.append(0)
        SUM_mkatedouble_cfos_low.append(0)
        rat_ns.append(rat)
        
    
RAM1011_mkatedouble_by_cfos = pd.DataFrame()
RAM1011_mkatedouble_by_cfos['Rat_n'] = rat_ns
RAM1011_mkatedouble_by_cfos['Stain'] = 'mKate2 coloc with c-Fos'
RAM1011_mkatedouble_by_cfos['Sum Raw Cell n, c-Fos high'] = SUM_mkatedouble_cfos_high
RAM1011_mkatedouble_by_cfos['Sum Raw Cell n, c-Fos low'] = SUM_mkatedouble_cfos_low

In [11]:
RAM1011_mkatedouble_by_cfos

,Rat_n,Stain,"Sum Raw Cell n, c-Fos high","Sum Raw Cell n, c-Fos low"
0,RAM-10_rat10C,mKate2 coloc with c-Fos,6,5
1,RAM-10_rat11C,mKate2 coloc with c-Fos,65,45
2,RAM-10_rat12C,mKate2 coloc with c-Fos,5,2
3,RAM-10_rat13C,mKate2 coloc with c-Fos,3,0
4,RAM-10_rat14C,mKate2 coloc with c-Fos,8,3
5,RAM-10_rat15C,mKate2 coloc with c-Fos,12,5
6,RAM-10_rat16C,mKate2 coloc with c-Fos,2,6
7,RAM-10_rat1C,mKate2 coloc with c-Fos,2,1
8,RAM-10_rat2C,mKate2 coloc with c-Fos,1,1
9,RAM-10_rat3C,mKate2 coloc with c-Fos,3,1


In [12]:
rats = np.unique(RAM1011.Rat_n)
treat = []
dox = []
react = []
for rat in rats:
    treat.append(np.unique(RAM1011[RAM1011.Rat_n == rat]['Treatment']).item())
    dox.append(np.unique(RAM1011[RAM1011.Rat_n == rat]['Dox']).item())
    react.append(np.unique(RAM1011[RAM1011.Rat_n == rat]['React']).item())
    
d_treat = dict(zip(rats, treat))
d_dox = dict(zip(rats, dox))
d_react = dict(zip(rats, react))

images = np.unique(['_'.join(f.split('_')[:-1]) for f in list(RAM1011.FileName)])
img_rat, img_count = np.unique(['_'.join(f.split('_')[:2]) for f in images], return_counts=True)

img_rat = ['_'.join([rat.split('_')[0], 'rat'+rat.split('_')[1]]) for rat in img_rat]
d_imgn = dict(zip(img_rat, img_count))

rats = []
treat = []
dox = []
react = []
img_n = []
mean_cell_n_cfoshigh = []
mean_cell_n_cfoslow = []
for rat in RAM1011_mkatedouble_by_cfos.Rat_n:
    treat.append(d_treat[rat])
    dox.append(d_dox[rat])
    react.append(d_react[rat])
    img_n.append(d_imgn[rat])    
    sum_n = np.array(RAM1011_mkatedouble_by_cfos[RAM1011_mkatedouble_by_cfos['Rat_n'] == rat]['Sum Raw Cell n, c-Fos high']).item()
    mean_cell_n_cfoshigh.append(sum_n / d_imgn[rat])
    sum_n = np.array(RAM1011_mkatedouble_by_cfos[RAM1011_mkatedouble_by_cfos['Rat_n'] == rat]['Sum Raw Cell n, c-Fos low']).item()
    mean_cell_n_cfoslow.append(sum_n / d_imgn[rat])

    rats.append(rat)
    
RAM1011_mkatedouble_by_cfos = RAM1011_mkatedouble_by_cfos.drop(columns=['Rat_n'])

RAM1011_mkatedouble_by_cfos['Total Image n'] = img_n
RAM1011_mkatedouble_by_cfos['Mean Cell n, c-Fos High'] = mean_cell_n_cfoshigh
RAM1011_mkatedouble_by_cfos['Mean Cell n, c-Fos Low'] = mean_cell_n_cfoslow
RAM1011_mkatedouble_by_cfos['Rat_n'] = rats
RAM1011_mkatedouble_by_cfos['Dox'] = dox
RAM1011_mkatedouble_by_cfos['Treatment'] = treat
RAM1011_mkatedouble_by_cfos['React'] = react

RAM1011_mkatedouble_by_cfos = RAM1011_mkatedouble_by_cfos.sort_values(by=['Dox', 'Treatment', 'React','Rat_n'])

RAM1011_mkatedouble_by_cfos

,Stain,"Sum Raw Cell n, c-Fos high","Sum Raw Cell n, c-Fos low",Total Image n,"Mean Cell n, c-Fos High","Mean Cell n, c-Fos Low",Rat_n,Dox,Treatment,React
5,mKate2 coloc with c-Fos,12,5,4,3.000000,1.250000,RAM-10_rat15C,DOX_OFF,ChABC,FR1
10,mKate2 coloc with c-Fos,46,47,6,7.666667,7.833333,RAM-10_rat5C,DOX_OFF,ChABC,FR1
18,mKate2 coloc with c-Fos,50,43,6,8.333333,7.166667,RAM-11_rat14,DOX_OFF,ChABC,FR1
19,mKate2 coloc with c-Fos,34,29,6,5.666667,4.833333,RAM-11_rat15,DOX_OFF,ChABC,FR1
9,mKate2 coloc with c-Fos,3,1,5,0.600000,0.200000,RAM-10_rat3C,DOX_OFF,ChABC,NR
20,mKate2 coloc with c-Fos,7,2,6,1.166667,0.333333,RAM-11_rat2,DOX_OFF,ChABC,NR
6,mKate2 coloc with c-Fos,2,6,5,0.400000,1.200000,RAM-10_rat16C,DOX_OFF,ChABC,VR5
11,mKate2 coloc with c-Fos,14,8,6,2.333333,1.333333,RAM-10_rat6C,DOX_OFF,ChABC,VR5
22,mKate2 coloc with c-Fos,28,27,6,4.666667,4.500000,RAM-11_rat4,DOX_OFF,ChABC,VR5
26,mKate2 coloc with c-Fos,34,48,6,5.666667,8.000000,RAM-11_rat8,DOX_OFF,ChABC,VR5


In [13]:
RAM1011_mkatedouble_by_cfos.to_csv('RAM-10,11 mKate2 coloc with c-Fos Mean Cell ns, c-Fos Median Split.csv')
pd.concat([RAM10_mkateoncfos_nearestcfos, RAM10_mkateoncfos_nearestcfos]).to_csv('RAM-10,11, mKate2 coloc with c-Fos, c-Fos Median Split RAW.csv')

In [14]:
set(RAM1011.Rat_n).symmetric_difference(set(RAM1011_mkatedouble_by_cfos.Rat_n))

set()

In [21]:
RAM10_cfos_median = np.median(RAM10_bystain_NORM[6]['Mean-Background_NORM'])
RAM11_cfos_median = np.median(RAM11_bystain_NORM[6]['Mean-Background_NORM'])

RAM10_cfos = RAM10_bystain_NORM[6].copy()
RAM11_cfos = RAM11_bystain_NORM[6].copy()

RAM10_cfos_highlow = []
for cfosint in list(RAM10_cfos['Mean-Background_NORM']):
    if cfosint < RAM10_cfos_median:
        RAM10_cfos_highlow.append('c-Fos low')
    elif cfosint >= RAM10_cfos_median:
        RAM10_cfos_highlow.append('c-Fos high')
        
RAM10_cfos['c-Fos Intensity Bin'] = RAM10_cfos_highlow
        
RAM11_cfos_highlow = []
for cfosint in list(RAM11_cfos['Mean-Background_NORM']):
    if cfosint < RAM11_cfos_median:
        RAM11_cfos_highlow.append('c-Fos low')
    elif cfosint >= RAM11_cfos_median:
        RAM11_cfos_highlow.append('c-Fos high')
        
RAM11_cfos['c-Fos Intensity Bin'] = RAM11_cfos_highlow

RAM1011_cfos = pd.concat([RAM10_cfos, RAM11_cfos])
RAM1011_cfos

,,AR,AnalysisDate,Area,Background,CellNumber,Circ.,Experiment,FileName,Integrated_Density,Max,Mean,Mean-Background,Min,Round,Solidity,Stain,StdDev,Subject,XM,YM,directory,roiName,Rat_n,Treatment,Dox,React,Coord,Mean-Background_NORM,c-Fos Intensity Bin
7833,1,1.169,_11_20_2022_at_15-35,65.935,39.644,1,0.985,RAM-10,RAM-10_rat10C_PFC_3.8_G,4069.323,191.202,101.361,61.717,40.506,0.837,0.959,c-Fos,37.078,10C,197.735,117.246,/Volumes/T7/RAM-10/summed/RAM-10_10C_PFC_3.8_G...,005a-00012,RAM-10_rat10C,ChABC,DOX_ON,NR,"(197.735, 117.246)",1.212336,c-Fos high
7834,2,1.000,_11_20_2022_at_15-35,68.040,39.644,2,1.000,RAM-10,RAM-10_rat10C_PFC_3.8_G,6193.684,264.734,130.674,91.030,50.056999999999995,1.000,0.960,c-Fos,44.663,10C,211.344,302.819,/Volumes/T7/RAM-10/summed/RAM-10_10C_PFC_3.8_G...,005a-00002,RAM-10_rat10C,ChABC,DOX_ON,NR,"(211.344, 302.819)",1.788146,c-Fos high
7835,3,1.000,_11_20_2022_at_15-35,52.608,39.644,3,0.955,RAM-10,RAM-10_rat10C_PFC_3.8_G,2162.829,158.994,80.756,41.112,39.909,0.938,0.893,c-Fos,26.053,10C,236.864,324.349,/Volumes/T7/RAM-10/summed/RAM-10_10C_PFC_3.8_G...,005a-00009,RAM-10_rat10C,ChABC,DOX_ON,NR,"(236.864, 324.349)",0.807583,c-Fos low
7836,4,1.092,_11_20_2022_at_15-35,41.385,39.644,4,0.831,RAM-10,RAM-10_rat10C_PFC_3.8_G,1046.948,145.040,64.942,25.298,39.986,0.772,0.797,c-Fos,21.149,10C,127.097,161.201,/Volumes/T7/RAM-10/summed/RAM-10_10C_PFC_3.8_G...,005a-00003,RAM-10_rat10C,ChABC,DOX_ON,NR,"(127.097, 161.201)",0.496941,c-Fos low
7837,5,1.169,_11_20_2022_at_15-35,60.324,39.644,5,0.901,RAM-10,RAM-10_rat10C_PFC_3.8_G,3382.353,172.732,95.714,56.070,42.952,0.766,0.878,c-Fos,32.792,10C,325.073,85.148,/Volumes/T7/RAM-10/summed/RAM-10_10C_PFC_3.8_G...,005a-00007,RAM-10_rat10C,ChABC,DOX_ON,NR,"(325.073, 85.148)",1.101410,c-Fos high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35328,122,1.000,_12_14_2022_at_10-48,52.608,7.702,122,0.955,RAM-11,RAM-11_rat6_PL_4.0_D,916.205,53.540,25.117,17.416,7.937,0.938,0.893,c-Fos,10.237,6,26.933,284.674,RAM-11_PFC/summed/RAM-11_6_PL_4.0_D_2/RAM-11_6...,0340a-0032,RAM-11_rat6,Vehicle,DOX_OFF,VR5,"(26.933000000000003, 284.67400000000004)",0.347750,c-Fos low
35329,123,1.000,_12_14_2022_at_10-48,52.608,7.702,123,0.955,RAM-11,RAM-11_rat6_PL_4.0_D,916.205,53.540,25.117,17.416,7.937,0.938,0.893,c-Fos,10.237,6,26.933,284.674,RAM-11_PFC/summed/RAM-11_6_PL_4.0_D_2/RAM-11_6...,0340a-0032,RAM-11_rat6,Vehicle,DOX_OFF,VR5,"(26.933000000000003, 284.67400000000004)",0.347750,c-Fos low
35330,124,1.111,_12_14_2022_at_10-48,33.669,7.702,124,0.844,RAM-11,RAM-11_rat6_PL_4.0_D,611.755,54.154,25.871,18.170,8.559,0.720,0.828,c-Fos,10.842,6,122.502,169.571,RAM-11_PFC/summed/RAM-11_6_PL_4.0_D_2/RAM-11_6...,0202a-0147,RAM-11_rat6,Vehicle,DOX_OFF,VR5,"(122.502, 169.571)",0.362805,c-Fos low
35331,125,1.302,_12_14_2022_at_10-48,38.579,7.702,125,0.859,RAM-11,RAM-11_rat6_PL_4.0_D,688.160,61.907,25.539,17.838,8.198,0.660,0.833,c-Fos,13.025,6,267.618,172.569,RAM-11_PFC/summed/RAM-11_6_PL_4.0_D_2/RAM-11_6...,0206a-0319,RAM-11_rat6,Vehicle,DOX_OFF,VR5,"(267.618, 172.56900000000002)",0.356176,c-Fos low


In [28]:
df = RAM1011_cfos

cfos_high_by_image = []
cfos_low_by_image = []
images = []
for rat in np.unique(df['Rat_n']):
    df_rat = df[df.Rat_n == rat]
    
    for image in np.unique(df_rat['FileName']):
        df_rat_image = df_rat[df_rat.FileName == image]
        cfos_high_by_image.append(len(df_rat_image[df_rat_image['c-Fos Intensity Bin'] == 'c-Fos high']))
        cfos_low_by_image.append(len(df_rat_image[df_rat_image['c-Fos Intensity Bin'] == 'c-Fos low']))
        images.append(image)

RAM1011_cfoshighlow_counts = pd.DataFrame()
RAM1011_cfoshighlow_counts['Image'] = images
RAM1011_cfoshighlow_counts['c-Fos high'] = cfos_high_by_image
RAM1011_cfoshighlow_counts['c-Fos low'] = cfos_low_by_image

RAM1011_cfoshighlow_counts

,Image,c-Fos high,c-Fos low
0,RAM-10_rat10C_PFC_3.8_G,12,80
1,RAM-10_rat10C_PFC_3.9_E,3,26
2,RAM-10_rat10C_PFC_3.9_F,9,53
3,RAM-10_rat10C_PFC_4.0_D,2,4
4,RAM-10_rat10C_PFC_4.2_C,15,7
...,...,...,...
152,RAM-11_rat9_PL_3.5_E,29,171
153,RAM-11_rat9_PL_3.6_C,43,174
154,RAM-11_rat9_PL_3.6_F,36,110
155,RAM-11_rat9_PL_3.7_A,24,47


In [32]:
rat_ns = []
for image in RAM1011_cfoshighlow_counts['Image']:
    rat_n = '_'.join(image.split('_')[0:2])
    rat_ns.append(rat_n)
    
RAM1011_cfoshighlow_counts['Rat_n'] = rat_ns

rats = np.unique(RAM1011.Rat_n)

SUM_cfos_high = []
SUM_cfos_low = []
rat_ns = []
for rat in rats:
    
    if rat in set(RAM1011_cfoshighlow_counts['Rat_n']):
        df = RAM1011_cfoshighlow_counts[RAM1011_cfoshighlow_counts['Rat_n'] == rat]
        SUM_cfos_high.append(np.sum(df['c-Fos high']))
        SUM_cfos_low.append(np.sum(df['c-Fos low']))
        rat_ns.append(rat)
        
    elif rat not in set(RAM1011_cfoshighlow_counts['Rat_n']):
        SUM_cfos_high.append(0)
        SUM_cfos_low.append(0)
        rat_ns.append(rat)
        
    
RAM1011_by_cfos = pd.DataFrame()
RAM1011_by_cfos['Rat_n'] = rat_ns
RAM1011_by_cfos['Stain'] = 'c-Fos'
RAM1011_by_cfos['Sum Raw Cell n, c-Fos high'] = SUM_cfos_high
RAM1011_by_cfos['Sum Raw Cell n, c-Fos low'] = SUM_cfos_low

RAM1011_by_cfos

,Rat_n,Stain,"Sum Raw Cell n, c-Fos high","Sum Raw Cell n, c-Fos low"
0,RAM-10_rat10C,c-Fos,86,189
1,RAM-10_rat11C,c-Fos,868,613
2,RAM-10_rat12C,c-Fos,170,52
3,RAM-10_rat13C,c-Fos,166,20
4,RAM-10_rat14C,c-Fos,572,192
5,RAM-10_rat15C,c-Fos,96,131
6,RAM-10_rat16C,c-Fos,105,372
7,RAM-10_rat1C,c-Fos,321,323
8,RAM-10_rat2C,c-Fos,295,372
9,RAM-10_rat3C,c-Fos,257,283


In [33]:
rats = np.unique(RAM1011.Rat_n)
treat = []
dox = []
react = []
for rat in rats:
    treat.append(np.unique(RAM1011[RAM1011.Rat_n == rat]['Treatment']).item())
    dox.append(np.unique(RAM1011[RAM1011.Rat_n == rat]['Dox']).item())
    react.append(np.unique(RAM1011[RAM1011.Rat_n == rat]['React']).item())
    
d_treat = dict(zip(rats, treat))
d_dox = dict(zip(rats, dox))
d_react = dict(zip(rats, react))

images = np.unique(['_'.join(f.split('_')[:-1]) for f in list(RAM1011.FileName)])
img_rat, img_count = np.unique(['_'.join(f.split('_')[:2]) for f in images], return_counts=True)

img_rat = ['_'.join([rat.split('_')[0], 'rat'+rat.split('_')[1]]) for rat in img_rat]
d_imgn = dict(zip(img_rat, img_count))

rats = []
treat = []
dox = []
react = []
img_n = []
mean_cell_n_cfoshigh = []
mean_cell_n_cfoslow = []
for rat in RAM1011_by_cfos.Rat_n:
    treat.append(d_treat[rat])
    dox.append(d_dox[rat])
    react.append(d_react[rat])
    img_n.append(d_imgn[rat])    
    sum_n = np.array(RAM1011_by_cfos[RAM1011_by_cfos['Rat_n'] == rat]['Sum Raw Cell n, c-Fos high']).item()
    mean_cell_n_cfoshigh.append(sum_n / d_imgn[rat])
    sum_n = np.array(RAM1011_by_cfos[RAM1011_by_cfos['Rat_n'] == rat]['Sum Raw Cell n, c-Fos low']).item()
    mean_cell_n_cfoslow.append(sum_n / d_imgn[rat])

    rats.append(rat)
    
RAM1011_by_cfos = RAM1011_by_cfos.drop(columns=['Rat_n'])

RAM1011_by_cfos['Total Image n'] = img_n
RAM1011_by_cfos['Mean Cell n, c-Fos High'] = mean_cell_n_cfoshigh
RAM1011_by_cfos['Mean Cell n, c-Fos Low'] = mean_cell_n_cfoslow
RAM1011_by_cfos['Rat_n'] = rats
RAM1011_by_cfos['Dox'] = dox
RAM1011_by_cfos['Treatment'] = treat
RAM1011_by_cfos['React'] = react

RAM1011_by_cfos = RAM1011_by_cfos.sort_values(by=['Dox', 'Treatment', 'React','Rat_n'])

RAM1011_by_cfos

,Stain,"Sum Raw Cell n, c-Fos high","Sum Raw Cell n, c-Fos low",Total Image n,"Mean Cell n, c-Fos High","Mean Cell n, c-Fos Low",Rat_n,Dox,Treatment,React
5,c-Fos,96,131,4,24.000000,32.750000,RAM-10_rat15C,DOX_OFF,ChABC,FR1
10,c-Fos,560,577,6,93.333333,96.166667,RAM-10_rat5C,DOX_OFF,ChABC,FR1
18,c-Fos,565,545,6,94.166667,90.833333,RAM-11_rat14,DOX_OFF,ChABC,FR1
19,c-Fos,338,343,6,56.333333,57.166667,RAM-11_rat15,DOX_OFF,ChABC,FR1
9,c-Fos,257,283,5,51.400000,56.600000,RAM-10_rat3C,DOX_OFF,ChABC,NR
20,c-Fos,503,44,6,83.833333,7.333333,RAM-11_rat2,DOX_OFF,ChABC,NR
6,c-Fos,105,372,5,21.000000,74.400000,RAM-10_rat16C,DOX_OFF,ChABC,VR5
11,c-Fos,348,612,6,58.000000,102.000000,RAM-10_rat6C,DOX_OFF,ChABC,VR5
22,c-Fos,503,471,6,83.833333,78.500000,RAM-11_rat4,DOX_OFF,ChABC,VR5
26,c-Fos,310,692,6,51.666667,115.333333,RAM-11_rat8,DOX_OFF,ChABC,VR5


In [35]:
RAM1011_by_cfos.to_csv('RAM-10,11 Single c-Fos Mean Cell ns, c-Fos Median Split.csv')
RAM1011_cfos.to_csv('RAM-10,11 Single c-Fos Median Split RAW.csv')